In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import math
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import MinMaxScaler

device = "cuda"

## Load Images

In [2]:
#---- required for med-clip ----
import numpy as np
import pandas as pd
#---- required for dataloader ----
from torch.utils.data import Dataset, DataLoader
import torch
from tqdm import tqdm
import os
#---- required for linear SVM ----
import random
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC
#---- Required for t-SNE ----
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE

In [3]:
# Define age group assignment function
def assign_age_group(age):
    if 55 <= age < 60:
        return '55-60'
    elif 60 <= age < 65:
        return '60-65'
    elif 65 <= age < 70:
        return '65-70'
    elif age >= 70:
        return '70+'
    else:
        return 'Under 50'  

def get_patient_data():
    #---- Load training embeddings ----
    train_data = np.load('nlst_train_with_labels.npz', allow_pickle=True)["arr_0"].item()
    test_data = np.load('nlst_tune_with_labels.npz', allow_pickle=True)["arr_0"].item()
    # construct dataframes
    train_df = pd.DataFrame.from_dict(train_data, orient='index')
    test_df = pd.DataFrame.from_dict(test_data, orient='index')
    # Acquire unique identifiers
    train_df["pid"] = [k.split('/')[1] for k in list(train_data.keys())]
    test_df["pid"] = [k.split('/')[1] for k in list(test_data.keys())]
    # Replace first row with indices
    train_df.reset_index(drop=True, inplace=True)
    test_df.reset_index(drop=True, inplace=True)

    #---- Load patient demographics ----
    df = pd.read_csv("nlst_780_prsn_idc_20210527.csv")
    df["gender"] = df["gender"].map({1:"M", 2:"F"})

    #---- add patient demographics to dataset ---- 
    train_df['pid'], test_df['pid'], df['pid'] = train_df['pid'].astype(str), test_df['pid'].astype(str), df['pid'].astype(str)
    train_df = train_df.merge(df[['pid', 'gender', "age", "race", "can_scr"]], on='pid', how='left')
    test_df = test_df.merge(df[['pid', 'gender', "age", "race", "can_scr"]], on='pid', how='left')
    # define age groups
    train_df['Age_group'], test_df['Age_group'] = train_df['age'].apply(assign_age_group), test_df['age'].apply(assign_age_group)
    return train_df, test_df

In [4]:
#---- Load training embeddings ----
train_data = np.load('nlst_train_with_labels.npz', allow_pickle=True)["arr_0"].item()
test_data = np.load('nlst_tune_with_labels.npz', allow_pickle=True)["arr_0"].item()
# construct dataframes
train_df = pd.DataFrame.from_dict(train_data, orient='index')
test_df = pd.DataFrame.from_dict(test_data, orient='index')
# Acquire unique identifiers
train_df["pid"] = [k.split('/')[1] for k in list(train_data.keys())]
test_df["pid"] = [k.split('/')[1] for k in list(test_data.keys())]
# Replace first row with indices
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

#---- Load patient demographics ----
df = pd.read_csv("nlst_780_prsn_idc_20210527.csv")
demo_df = df[["pid","race", "gender", "age", "can_scr"]]
demo_df["gender"] = demo_df["gender"].map({1:"M", 2:"F"})

#---- add patient demographics to dataset ---- 
train_df['pid'], test_df['pid'], demo_df['pid'] = train_df['pid'].astype(str), test_df['pid'].astype(str), demo_df['pid'].astype(str)
train_df = train_df.merge(demo_df[['pid', 'gender', "age", "race", "can_scr"]], on='pid', how='left')
test_df = test_df.merge(demo_df[['pid', 'gender', "age", "race", "can_scr"]], on='pid', how='left')

/tmp/ipykernel_18767/3017132832.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demo_df["gender"] = demo_df["gender"].map({1:"M", 2:"F"})
/tmp/ipykernel_18767/3017132832.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['pid'], test_df['pid'], demo_df['pid'] = train_df['pid'].astype(str), test_df['pid'].astype(str), demo_df['pid'].astype(str)


In [5]:
def scale_datasets(x_train, x_test):
    """
    Standard Scale test and train data
    """
    standard_scaler = MinMaxScaler()
    x_train_scaled = standard_scaler.fit_transform(x_train)
    x_test_scaled = standard_scaler.transform(x_test)
    return x_train_scaled, x_test_scaled

In [6]:
#---- Load train/ test dataset ----
X_train = np.vstack(list(train_df["embedding"]))
X_test = np.array(list(test_df["embedding"]))
y_train, y_test = train_df["cancer_in_2"].values,  test_df["cancer_in_2"].values

x_train_scaled, x_test_scaled = scale_datasets(X_train, X_test)
print("X shape: ", x_train_scaled.shape, "y shape: ", y_train.shape)

x_train_tensor = torch.tensor(x_train_scaled, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test_scaled, dtype=torch.float32)

# Dataset
train_dataset = TensorDataset(x_train_tensor, x_train_tensor)
test_dataset = TensorDataset(x_test_tensor, x_test_tensor)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Check shape
for (batch_idx, train_tensors) in enumerate(train_loader):
    print(train_tensors[0].shape)  # --> torch.Size([32, 1408])
    break

X shape:  (52696, 1408) y shape:  (52696,)
torch.Size([32, 1408])


## Define VQVAE

In [7]:
#!wget https://raw.githubusercontent.com/airalcorn2/vqvae-pytorch/master/vqvae.py

In [14]:
import numpy as np
import torch

from PIL import Image
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import CIFAR10
from vqvae1D import VQVAE

torch.set_printoptions(linewidth=160)


def display_img_tensors_as_grid(img_tensors, nrows, f):
    img_tensors = img_tensors.permute(0, 2, 3, 1)
    imgs_array = img_tensors.detach().cpu().numpy()
    imgs_array[imgs_array < -0.5] = -0.5
    imgs_array[imgs_array > 0.5] = 0.5
    imgs_array = 255 * (imgs_array + 0.5)
    (batch_size, img_size) = img_tensors.shape[:2]
    ncols = batch_size // nrows
    img_arr = np.zeros((nrows * batch_size, ncols * batch_size, 3))
    for idx in range(batch_size):
        row_idx = idx // ncols
        col_idx = idx % ncols
        row_start = row_idx * img_size
        row_end = row_start + img_size
        col_start = col_idx * img_size
        col_end = col_start + img_size
        img_arr[row_start:row_end, col_start:col_end] = imgs_array[idx]

    display(Image.fromarray(img_arr.astype(np.uint8), "RGB"))

In [12]:
# Initialize model.
device = torch.device("cuda:0")
use_ema = True
model_args = {
    "in_channels": 1,
    "num_hiddens": 128,
    "num_downsampling_layers": 2,
    "num_residual_layers": 2,
    "num_residual_hiddens": 32,
    "embedding_dim": 64,
    "num_embeddings": 512,
    "use_ema": use_ema,
    "decay": 0.99,
    "epsilon": 1e-5,
}
model = VQVAE(**model_args).to(device)

# Initialize dataset.
batch_size = 16
workers = 10
normalize = transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[1.0, 1.0, 1.0])
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        normalize,
    ]
)
data_root = "data"
# train_dataset = CIFAR10(data_root, True, transform, download=True)
train_data_variance = np.var(x_train_scaled / 255)
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=workers,
)

# Multiplier for commitment loss. See Equation (3) in "Neural Discrete Representation
# Learning".
beta = 0.25

# Initialize optimizer.
train_params = [params for params in model.parameters()]
lr = 3e-4
optimizer = optim.Adam(train_params, lr=lr)
criterion = nn.MSELoss()

# Train model.
epochs = 50
eval_every = 200
best_train_loss = float("inf")
model.train()

VQVAE(
  (encoder): Encoder(
    (conv): Sequential(
      (down0): Conv1d(1, 64, kernel_size=(4,), stride=(2,), padding=(1,))
      (relu0): ReLU()
      (down1): Conv1d(64, 128, kernel_size=(4,), stride=(2,), padding=(1,))
      (relu1): ReLU()
      (final_conv): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    )
    (residual_stack): ResidualStack(
      (layers): ModuleList(
        (0-1): 2 x Sequential(
          (0): ReLU()
          (1): Conv1d(128, 32, kernel_size=(3,), stride=(1,), padding=(1,))
          (2): ReLU()
          (3): Conv1d(32, 128, kernel_size=(1,), stride=(1,))
        )
      )
    )
  )
  (pre_vq_conv): Conv1d(128, 64, kernel_size=(1,), stride=(1,))
  (vq): VectorQuantizer(
    (N_i_ts): SonnetExponentialMovingAverage()
    (m_i_ts): SonnetExponentialMovingAverage()
  )
  (decoder): Decoder(
    (conv): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    (residual_stack): ResidualStack(
      (layers): ModuleList(
        (0-1

In [13]:
for epoch in range(epochs):
    total_train_loss = 0
    total_recon_error = 0
    n_train = 0
    for (batch_idx, train_tensors) in enumerate(train_loader):
        optimizer.zero_grad()
        imgs = train_tensors[0].unsqueeze(1).to(device)
        out = model(imgs)
        recon_error = criterion(out["x_recon"], imgs) / train_data_variance
        total_recon_error += recon_error.item()
        loss = recon_error + beta * out["commitment_loss"]
        if not use_ema:
            loss += out["dictionary_loss"]

        total_train_loss += loss.item()
        loss.backward()
        optimizer.step()
        n_train += 1

        if ((batch_idx + 1) % eval_every) == 0:
            print(f"epoch: {epoch}\nbatch_idx: {batch_idx + 1}", flush=True)
            total_train_loss /= n_train
            if total_train_loss < best_train_loss:
                best_train_loss = total_train_loss

            print(f"total_train_loss: {total_train_loss}")
            print(f"best_train_loss: {best_train_loss}")
            print(f"recon_error: {total_recon_error / n_train}\n")

            total_train_loss = 0
            total_recon_error = 0
            n_train = 0

epoch: 0
batch_idx: 500
total_train_loss: 348782.023609375
best_train_loss: 348782.023609375
recon_error: 348205.982703125

epoch: 0
batch_idx: 1000
total_train_loss: 5109493.7503125
best_train_loss: 348782.023609375
recon_error: 5049845.370875

epoch: 0
batch_idx: 1500
total_train_loss: 25870891.891
best_train_loss: 348782.023609375
recon_error: 25482115.182

epoch: 0
batch_idx: 2000
total_train_loss: 76133516.328
best_train_loss: 348782.023609375
recon_error: 74802314.024

epoch: 0
batch_idx: 2500
total_train_loss: 222020987.712
best_train_loss: 348782.023609375
recon_error: 217136639.2

epoch: 0
batch_idx: 3000
total_train_loss: 365490919.936
best_train_loss: 348782.023609375
recon_error: 349068052.448

epoch: 1
batch_idx: 500
total_train_loss: 1596087290.88
best_train_loss: 348782.023609375
recon_error: 1524191638.016

epoch: 1
batch_idx: 1000
total_train_loss: 2842690000.64
best_train_loss: 348782.023609375
recon_error: 2720599952.896

epoch: 1
batch_idx: 1500
total_train_loss: 44

KeyboardInterrupt: 

In [ ]:
# VAE definition
class VAE(nn.Module):
    def __init__(self, input_dim, latent_dim=64):
        super(VAE, self).__init__()
        # Encoder layers
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU()
        )
        self.fc_mu = nn.Linear(128, latent_dim)
        self.fc_logvar = nn.Linear(128, latent_dim)

        # Decoder layers
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, input_dim),
            nn.Sigmoid()  # because you scaled inputs to [0,1]
        )

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        h = self.encoder(x)
        mu, logvar = self.fc_mu(h), self.fc_logvar(h)
        z = self.reparameterize(mu, logvar)
        recon_x = self.decoder(z)
        return recon_x, mu, logvar


# VAE loss (Reconstruction + KL Divergence)
def vae_loss(recon_x, x, mu, logvar):
    recon_loss = nn.functional.mse_loss(recon_x, x, reduction='sum')  # or L1 if you prefer
    # KL divergence
    kld = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return recon_loss + kld

In [ ]:
# ---- Training loop ----
output_units = x_train_scaled.shape[1]
latent_dim = 64
model = VAE(input_dim=output_units, latent_dim=latent_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.00001)

num_epochs = 200
train_losses, val_losses = [], []
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for data, _ in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = vae_loss(recon_batch, data, mu, logvar)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader.dataset)
    train_losses.append(train_loss)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for data, _ in test_loader:
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            loss = vae_loss(recon_batch, data, mu, logvar)
            val_loss += loss.item()
    val_loss /= len(test_loader.dataset)
    val_losses.append(val_loss)

    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

In [ ]:
# ---- Extract embeddings (mean of latent space) ----
with torch.no_grad():
    mu_train, _ = model.fc_mu(model.encoder(x_train_tensor.to(device))), model.fc_logvar(model.encoder(x_train_tensor.to(device)))
    mu_test, _  = model.fc_mu(model.encoder(x_test_tensor.to(device))),  model.fc_logvar(model.encoder(x_test_tensor.to(device)))
train, test = mu_train.cpu().numpy(), mu_test.cpu().numpy()

#---- Acquire Patient Data ----
recon_train_df, recon_test_df = train_df.drop('embedding', axis=1), test_df.drop('embedding', axis=1)
recon_train_df["embedding"], recon_test_df["embedding"] = [row.tolist() for row in train], [row.tolist() for row in test]

#---- Save Low Dimensional Embeddings ----
reduced_train_df.to_csv(f'{savepath}/train_nlst_{dim}.csv', index=False); reduced_test_df.to_csv(f'{savepath}/test_nlst_{dim}.csv', index=False) 
print(f"{dim} dimensional NLST embeddings saved to {savepath}")